In [56]:
import sklearn.linear_model
import numpy as np
import pandas as pd
import os
from SoundLights.dataset.features_groups import  ARAUS_features, Freesound_features, mix_features, masker_features
from SoundLights.models.models_functions import clip, normalize_columns, normalize_columns_minmax

from sklearn.preprocessing import RobustScaler
import numpy as np


In [57]:
def prepare_data_models(dataframe, features_evaluated, maskers_gain: float = 5):

    # Drop string columns
    """dataframe = dataframe.drop("info.file", axis=1)
    dataframe = dataframe.drop("info.participant", axis=1)"""

    # Maskers colum, increase values
    dataframe["info.masker_bird"] = dataframe["info.masker_bird"] * maskers_gain
    dataframe["info.masker_construction"] = (
        dataframe["info.masker_construction"] * maskers_gain
    )
    dataframe["info.masker_traffic"] = dataframe["info.masker_traffic"] * maskers_gain
    dataframe["info.masker_water"] = dataframe["info.masker_water"] * maskers_gain
    dataframe["info.masker_wind"] = dataframe["info.masker_wind"] * maskers_gain

    # For fold 0, group data
    dataframe_fold0 = dataframe[dataframe["info.fold"] == 0]
    # Drop string columns
    dataframe_fold0 = dataframe_fold0.drop("info.file", axis=1)
    dataframe_fold0 = dataframe_fold0.drop("info.participant", axis=1)
    dataframe_fold0 = dataframe_fold0.groupby(
        ["info.soundscape", "info.masker", "info.smr"]
    ).mean()  # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth
    dataframe_filtered = dataframe[
        dataframe["info.fold"] != 0
    ]  # Filter rows where 'fold' column is not equal to 0
    dataframe = pd.concat(
        [dataframe_fold0, dataframe_filtered], ignore_index=True
    )  # Join together

    # Drop columns with all equal values or std=0
    std = np.std(dataframe[features_evaluated], axis=0)
    columns_to_mantain_arg = np.where(std >= 0.00001)[0]
    columns_to_drop_arg = np.where(std <= 0.00001)[0]
    columns_to_mantain = [features_evaluated[i] for i in columns_to_mantain_arg]
    columns_to_drop = [features_evaluated[i] for i in columns_to_drop_arg]
    # print(features_evaluated[np.where(std == 0)[0]])
    dataframe.drop(columns=columns_to_drop, inplace=True)

    return dataframe, columns_to_mantain



In [63]:
responses_Freesound= pd.read_csv(os.path.join('..','data/main_files','SoundLights_Freesound.csv')) #, dtype = {'participant':str}
responses_Freesound, features=prepare_data_models(responses_Freesound, Freesound_features)
print(responses_Freesound)


responses_Freesound= pd.read_csv(os.path.join('..','data/main_files','SoundLights_ARAUS.csv')) #, dtype = {'participant':str}
responses_Freesound, features=prepare_data_models(responses_Freesound, ARAUS_features, 1)
#features=ARAUS_features
print(responses_Freesound)


       info.fold  info.stimulus_index  info.wav_gain  info.time_taken  \
0            0.0                 27.4           6.44          48.5390   
1            0.0                 19.4           6.44          58.2362   
2            0.0                 28.4           6.44          45.8852   
3            0.0                 26.0           6.44          48.5338   
4            0.0                 17.4           6.44          53.0804   
...          ...                  ...            ...              ...   
25243        5.0                 40.0           6.44         101.2900   
25244        5.0                 41.0           6.44          63.4210   
25245        5.0                 42.0           6.44          83.9920   
25246        5.0                 43.0           6.44          74.0230   
25247        5.0                 44.0           6.44          62.8560   

       info.is_attention  info.pleasant  info.eventful  info.chaotic  \
0                    0.0            2.8            

### Adjust alpha

In [64]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1] 
l1_ratio = 0.5
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')

for value in alpha:

    model = ElasticNet(alpha=value, l1_ratio=l1_ratio, selection="random")
    #print(f'Investigating performance of {model} model...')

    MSEs_train = []
    MSEs_val = []
    MSEs_test = []
    MEs_train = []
    MEs_val = []
    MEs_test = []

    
    for val_fold in [1,2,3,4,5]:
        
        # Extract dataframes
        df_train = responses_Freesound[(responses_Freesound['info.fold'] != val_fold) & (responses_Freesound['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
        df_val   = responses_Freesound[responses_Freesound['info.fold'] == val_fold]
        df_test  = responses_Freesound[responses_Freesound['info.fold'] == 0]#.groupby(['info.soundscape','info.masker','info.smr']).mean() # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

        # Get ground-truth labels
        Y_train = df_train['info.P_ground_truth'].values
        Y_val = df_val['info.P_ground_truth'].values
        Y_test = df_test['info.P_ground_truth'].values


        # Get features normalized_data = (data - min) / (max-min)
        X_train = df_train[features].values
        X_val =df_val[features].values
        X_test = df_test[features].values

        """ X_train, mean, std=normalize_columns(X_train)
        X_val= (X_val - mean) / (std)
        X_test= (X_test - mean) / (std) """
        """ X_train, min, max=normalize_columns_minmax(X_train)
        X_val= (X_val - min) / (max - min)
        X_test= (X_test - min) / (max - min) """

        # Fit model
        X_LR = model.fit(X_train, Y_train)

        # Get MSEs
        MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
        MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
        MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
        ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
        ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
        ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

        # Add metrics
        MSEs_train.append(MSE_train)
        MSEs_val.append(MSE_val)
        MSEs_test.append(MSE_test)
        MEs_train.append(ME_train)
        MEs_val.append(ME_val)
        MEs_test.append(ME_test)

        #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
    print("Parameters ",value, l1_ratio )
    print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  0.1 0.5
Mean | 0.1323 | 0.1347 | 0.0753 | 0.2970 | 0.2998 | 0.2331 |
Parameters  0.2 0.5
Mean | 0.1346 | 0.1368 | 0.0770 | 0.3005 | 0.3028 | 0.2331 |
Parameters  0.3 0.5
Mean | 0.1357 | 0.1376 | 0.0790 | 0.3022 | 0.3043 | 0.2343 |
Parameters  0.4 0.5
Mean | 0.1370 | 0.1387 | 0.0810 | 0.3041 | 0.3061 | 0.2358 |
Parameters  0.5 0.5
Mean | 0.1386 | 0.1402 | 0.0833 | 0.3062 | 0.3081 | 0.2381 |
Parameters  0.6 0.5
Mean | 0.1402 | 0.1415 | 0.0859 | 0.3084 | 0.3099 | 0.2410 |
Parameters  0.7 0.5
Mean | 0.1418 | 0.1426 | 0.0880 | 0.3104 | 0.3113 | 0.2434 |
P

### Adjust l1_ratio

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = 1
l1_ratio = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] #0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')

for value in l1_ratio:

    model = ElasticNet(alpha=alpha, l1_ratio=value, selection="random")
    #print(f'Investigating performance of {model} model...')

    MSEs_train = []
    MSEs_val = []
    MSEs_test = []
    MEs_train = []
    MEs_val = []
    MEs_test = []

    
    for val_fold in [1,2,3,4,5]:

        # Extract dataframes
        df_train = responses_Freesound[(responses_Freesound['info.fold'] != val_fold) & (responses_Freesound['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
        df_val   = responses_Freesound[responses_Freesound['info.fold'] == val_fold]
        df_test  = responses_Freesound[responses_Freesound['info.fold'] == 0] # For the test set, the same 48 stimuli were shown to all participants so we take the mean of their ratings as the ground truth

        # Get ground-truth labels
        Y_train = df_train['info.P_ground_truth'].values
        Y_val = df_val['info.P_ground_truth'].values
        Y_test = df_test['info.P_ground_truth'].values

        # Get features normalized_data = (data - min) / (max-min)
        X_train = df_train[features].values
        X_val =df_val[features].values
        X_test = df_test[features].values

        """ X_train, mean, std=normalize_columns(X_train)
        X_val= (X_val - mean) / (std)
        X_test= (X_test - mean) / (std) """
        """ X_train, min, max=normalize_columns_minmax(X_train)
        X_val= (X_val - min) / (max - min)
        X_test= (X_test - min) / (max - min) """

        # Fit model
        X_LR = model.fit(X_train, Y_train)

        # Get MSEs
        MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
        MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
        MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
        ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
        ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
        ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

        # Add metrics
        MSEs_train.append(MSE_train)
        MSEs_val.append(MSE_val)
        MSEs_test.append(MSE_test)
        MEs_train.append(ME_train)
        MEs_val.append(ME_val)
        MEs_test.append(ME_test)

        #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
    print("Parameters ",alpha, value )
    print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')
    """ coefficients = X_LR.coef_
    feature_importances = list(zip(ARAUS_features+masker_features, coefficients))
    feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
    list_features=[]
    for feature, importance in feature_importances:
        if float(importance)!=0:
            print(f"{feature}: {importance}")
            list_features.append(feature)
    print("LIST ",list_features) """

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------
Parameters  1 0.1
Mean | 0.1347 | 0.1368 | 0.1533 | 0.3006 | 0.3029 | 0.3177 |
Parameters  1 0.2
Mean | 0.1371 | 0.1388 | 0.1572 | 0.3042 | 0.3062 | 0.3205 |
Parameters  1 0.3
Mean | 0.1402 | 0.1415 | 0.1621 | 0.3084 | 0.3099 | 0.3252 |
Parameters  1 0.4
Mean | 0.1426 | 0.1432 | 0.1657 | 0.3115 | 0.3121 | 0.3290 |
Parameters  1 0.5
Mean | 0.1435 | 0.1440 | 0.1681 | 0.3128 | 0.3134 | 0.3321 |
Parameters  1 0.6
Mean | 0.1445 | 0.1449 | 0.1706 | 0.3142 | 0.3147 | 0.3355 |
Parameters  1 0.7
Mean | 0.1455 | 0.1459 | 0.1733 | 0.3158 | 0.3161 | 0.3389 |
Parameters  1 0

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
# Suppress ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from sklearn.linear_model import ElasticNet
# Define your ElasticNet model with specific hyperparameters
alpha = 0.2
l1_ratio = 0.5
print('     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ ')
print('Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-')
print('     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures ')
print('-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------')



model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, selection="random")
#print(f'Investigating performance of {model} model...')

MSEs_train = []
MSEs_val = []
MSEs_test = []
MEs_train = []
MEs_val = []
MEs_test = []


for val_fold in [1,2,3,4,5]:

    # Extract dataframes
    df_train = responses_Freesound[(responses_Freesound['info.fold'] != val_fold) & (responses_Freesound['info.fold'] > 0)] # For the training set, use all samples that are not in the test set (fold 0) and current validation fold.
    df_val   = responses_Freesound[responses_Freesound['info.fold'] == val_fold]
    df_test  = responses_Freesound[responses_Freesound['info.fold'] == 0]

    # Get ground-truth labels
    Y_train = df_train['info.P_ground_truth'].values
    Y_val = df_val['info.P_ground_truth'].values
    Y_test = df_test['info.P_ground_truth'].values

    # Get features normalized_data = (data - min) / (max-min)
    X_train = df_train[Freesound_features+masker_features].values
    X_val =df_val[Freesound_features+masker_features].values
    X_test = df_test[Freesound_features+masker_features].values

    # Fit model
    X_LR = model.fit(X_train, Y_train)

    # Get MSEs
    MSE_train = np.mean((clip(X_LR.predict(X_train)) - Y_train)**2)
    MSE_val = np.mean((clip(X_LR.predict(X_val)) - Y_val)**2)
    MSE_test = np.mean((clip(X_LR.predict(X_test)) - Y_test)**2)
    ME_train = np.mean(np.abs(clip(X_LR.predict(X_train)) - Y_train))
    ME_val = np.mean(np.abs(clip(X_LR.predict(X_val)) - Y_val))
    ME_test = np.mean(np.abs(clip(X_LR.predict(X_test)) - Y_test))

    # Add metrics
    MSEs_train.append(MSE_train)
    MSEs_val.append(MSE_val)
    MSEs_test.append(MSE_test)
    MEs_train.append(ME_train)
    MEs_val.append(ME_val)
    MEs_test.append(ME_test)

    #print(f'{val_fold:4d} | {MSE_train:.4f} | {MSE_val:.4f} | {MSE_test:.4f} | {ME_train:.4f} | {ME_val:.4f} | {ME_test:.4f} | {X_LR.intercept_:7.4f} | {X_train.shape[0]:5d} | {X_val.shape[0]:5d} | {X_test.shape[0]:^4d} | {X_train.shape[1]:^5d} | {np.sum(np.abs(X_LR.coef_) > 0):^5d} |')
print("Parameters ",alpha, l1_ratio )
print(f'Mean | {np.mean(MSEs_train):.4f} | {np.mean(MSEs_val):.4f} | {np.mean(MSEs_test):.4f} | {np.mean(MEs_train):.4f} | {np.mean(MEs_val):.4f} | {np.mean(MEs_test):.4f} |')
coefficients = X_LR.coef_
feature_importances = list(zip(Freesound_features+masker_features, coefficients))
feature_importances.sort(key=lambda x: abs(x[1]), reverse=True)
list_features=[]
for feature, importance in feature_importances:
    if float(importance)!=0:
        print(f"{feature}: {importance}")
        list_features.append(feature)
print("LIST ",list_features)

     |    Mean squared error    |        Mean  error       |         |       # samples      | #     | # NZ 
Fold |--------+--------+--------|--------+--------+--------| Inter-  |-------+-------+------| feat- | feat-
     | Train  |   Val  |  Test  | Train  |   Val  |  Test  |  cept   | Train |  Val  | Test | ures  | ures 
-----+--------+--------+--------+--------+--------+--------+---------+-------+-------+------+-------+------


KeyError: "['freesound.lowlevel.average_loudness', 'freesound.lowlevel.dynamic_complexity', 'freesound.lowlevel.loudness_ebu128.avg', 'freesound.lowlevel.loudness_ebu128.var', 'freesound.lowlevel.loudness_ebu128.p20', 'freesound.lowlevel.loudness_ebu128.p80', 'freesound.lowlevel.spectral_flatness_db.avg', 'freesound.lowlevel.spectral_flatness_db.var', 'freesound.lowlevel.spectral_flatness_db.p20', 'freesound.lowlevel.spectral_flatness_db.p80', 'freesound.lowlevel.spectral_rolloff.avg', 'freesound.lowlevel.spectral_rolloff.var', 'freesound.lowlevel.spectral_rolloff.p20', 'freesound.lowlevel.spectral_rolloff.p80', 'freesound.lowlevel.spectral_flux.avg', 'freesound.lowlevel.spectral_flux.var', 'freesound.lowlevel.spectral_flux.p20', 'freesound.lowlevel.spectral_flux.p80', 'freesound.lowlevel.spectral_energyband_low.avg', 'freesound.lowlevel.spectral_energyband_low.var', 'freesound.lowlevel.spectral_energyband_low.p20', 'freesound.lowlevel.spectral_energyband_low.p80', 'freesound.lowlevel.